In [1]:
#import necessary libraries
import torch
from transformers import *
import pandas as pd
import re
import collections
import numpy as np
import json
import time
from tqdm.notebook import tqdm
import torch.nn as nn
import pathlib

#output all items, not just last one
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

#set device
if torch.cuda.is_available():
    device= "cuda"
else:
    device = "cpu"  
device

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


'cuda'

In [2]:
# define Articles dataset class for easy sampling, iteration, and weight creating
class Articles(torch.utils.data.Dataset):
    def __init__(self, json_file):
        super().__init__()
        with open(json_file, "r") as data_file:
            self.examples = json.loads(data_file.read())

    def __getitem__(self, idx):
        return self.examples[idx]

    def __len__(self):
        return len(self.examples)

    def tokenize(self):
        for idx, example in enumerate(self.examples):
            self.examples[idx]['text'] = re.findall('[\w]+', self.examples[idx]['text'].lower())

    def create_positive_sampler(self, target_publication):
        prob = np.zeros(len(self))
        for idx, example in enumerate(self.examples):
            if example['model_publication'] == target_publication:
                prob[idx] = 1
        return torch.utils.data.WeightedRandomSampler(weights=prob, num_samples=len(self), replacement=True)

    def create_negative_sampler(self, target_publication):
        prob = np.zeros(len(self))
        for idx, example in enumerate(self.examples):
            if example['model_publication'] != target_publication:
                prob[idx] = 1
        return torch.utils.data.WeightedRandomSampler(weights=prob, num_samples=len(self), replacement=True)

    def map_items(self, word_to_id, url_to_id, publication_to_id, filter=False, min_length=0):
        min_length_articles = []
        for idx, example in enumerate(self.examples):
            self.examples[idx]['text'] = [word_to_id.get(word, len(word_to_id)) for word in example['text']]
            self.examples[idx]['text'] = [word for word in example['text'] if word != len(word_to_id)]
            if filter:
                if len(self.examples[idx]['text']) > min_length:
                    min_length_articles.append(self.examples[idx])
            self.examples[idx]['url'] = url_to_id.get(example['url'], url_to_id.get("miscellaneous"))
            self.examples[idx]['model_publication'] = publication_to_id.get(example['model_publication'], publication_to_id.get("miscellaneous"))
        return min_length_articles


In [1]:
from transformers import LongformerTokenizer

# Load the BERT tokenizer.
print('Loading Longformer tokenizer...')
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

Loading Longformer tokenizer...


In [2]:
sentences = "The White House medical unit and Secret Service will evaluate attendees before they're admitted. They'll be required to test negative for the virus on the day of the event, complete a health questionnaire and pass a temperature screening. The sites will be cleaned and sanitized before each event. Trump Victory, the president’s joint fundraising committee, will cover the testing costs. The move comes as Trump expresses his desire for the country to reopen even as medical professionals raise concerns about the potential dangers of doing so. The pandemic death toll passed 100,000 this week. Trump is also dead set on holding the GOP’s August national convention in Charlotte, even as North Carolina officials are raising concerns about safety. The Charlotte area has seen an uptick in coronavirus cases in recent days. Trump has also been itching to resume his trademark rallies, his primary method of connecting with supporters and broadcasting his message. The president’s campaign team has used the events to glean data from attendees which is used to turn out his voters. The president has left the confines of the White House over the past few weeks to hold ostensibly official events in swing states like Arizona and Michigan, where polls have shown him trailing presumptive Democratic nominee Joe Biden. The events have sometimes had the feel of a rally, complete with walk-out music. Trump’s advisers want him to be seen as eager to reopen the country while Democrats push for stay-at-home orders that keep the economy shuttered. Earlier this month, the reelection effort released a one-minute advertisement titled “American Comeback” highlighting the president’s desire to reignite the economy. The spot concluded with Trump’s mantra that he will “make America great again.” The president’s previously busy fundraising schedule came to a halt in March. A planned March 12 fundraiser with GOP megadonor Sheldon Adelson was scrapped, as were a pair of events that month to be headlined by first lady Melania Trump. Even with in-person fundraising events slashed, the president has maintained robust fundraising totals thanks to a massive small-donor operation. Trump’s political machine narrowly outraised Biden in April and has a $187 million cash-on-hand lead over Biden and the Democratic Party. Biden has sworn off in-person fundraisers during the pandemic, instead doing online events from his Delaware home. Couples will need to donate $580,600 to attend the Dallas fundraiser. A single attendee to the New Jersey fundraiser will need to give $250,000. The money will go to Trump Victory, a joint fundraising committee of the Trump campaign, Republican National Committee, and state parties."
print(' Original: ', sentences)

 Original:  The White House medical unit and Secret Service will evaluate attendees before they're admitted. They'll be required to test negative for the virus on the day of the event, complete a health questionnaire and pass a temperature screening. The sites will be cleaned and sanitized before each event. Trump Victory, the president’s joint fundraising committee, will cover the testing costs. The move comes as Trump expresses his desire for the country to reopen even as medical professionals raise concerns about the potential dangers of doing so. The pandemic death toll passed 100,000 this week. Trump is also dead set on holding the GOP’s August national convention in Charlotte, even as North Carolina officials are raising concerns about safety. The Charlotte area has seen an uptick in coronavirus cases in recent days. Trump has also been itching to resume his trademark rallies, his primary method of connecting with supporters and broadcasting his message. The president’s campaign 

In [3]:
# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences))

Tokenized:  ['The', 'ĠWhite', 'ĠHouse', 'Ġmedical', 'Ġunit', 'Ġand', 'ĠSecret', 'ĠService', 'Ġwill', 'Ġevaluate', 'Ġattendees', 'Ġbefore', 'Ġthey', "'re", 'Ġadmitted', '.', 'ĠThey', "'ll", 'Ġbe', 'Ġrequired', 'Ġto', 'Ġtest', 'Ġnegative', 'Ġfor', 'Ġthe', 'Ġvirus', 'Ġon', 'Ġthe', 'Ġday', 'Ġof', 'Ġthe', 'Ġevent', ',', 'Ġcomplete', 'Ġa', 'Ġhealth', 'Ġquestionnaire', 'Ġand', 'Ġpass', 'Ġa', 'Ġtemperature', 'Ġscreening', '.', 'ĠThe', 'Ġsites', 'Ġwill', 'Ġbe', 'Ġcleaned', 'Ġand', 'Ġsan', 'itized', 'Ġbefore', 'Ġeach', 'Ġevent', '.', 'ĠTrump', 'ĠVictory', ',', 'Ġthe', 'Ġpresident', 'âĢ', 'Ļ', 's', 'Ġjoint', 'Ġfundraising', 'Ġcommittee', ',', 'Ġwill', 'Ġcover', 'Ġthe', 'Ġtesting', 'Ġcosts', '.', 'ĠThe', 'Ġmove', 'Ġcomes', 'Ġas', 'ĠTrump', 'Ġexpresses', 'Ġhis', 'Ġdesire', 'Ġfor', 'Ġthe', 'Ġcountry', 'Ġto', 'Ġreopen', 'Ġeven', 'Ġas', 'Ġmedical', 'Ġprofessionals', 'Ġraise', 'Ġconcerns', 'Ġabout', 'Ġthe', 'Ġpotential', 'Ġdangers', 'Ġof', 'Ġdoing', 'Ġso', '.', 'ĠThe', 'Ġpand', 'emic', 'Ġdeath', 'Ġtoll

In [7]:
from transformers import RobertaTokenizer

for idx, token in enumerate(tokenizer.vocab.keys()):
    print(token, " : ", idx)

AttributeError: 'LongformerTokenizer' object has no attribute 'vocab'

In [5]:
# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences)))

Token IDs:  [133, 735, 446, 1131, 1933, 8, 8350, 1841, 40, 10516, 10703, 137, 51, 214, 2641, 4, 252, 581, 28, 1552, 7, 1296, 2430, 13, 5, 6793, 15, 5, 183, 9, 5, 515, 6, 1498, 10, 474, 31067, 8, 1323, 10, 5181, 7231, 4, 20, 3091, 40, 28, 17317, 8, 15610, 37174, 137, 349, 515, 4, 140, 15908, 6, 5, 394, 17, 27, 29, 2660, 8023, 1540, 6, 40, 1719, 5, 3044, 1042, 4, 20, 517, 606, 25, 140, 15994, 39, 4724, 13, 5, 247, 7, 13490, 190, 25, 1131, 5197, 1693, 1379, 59, 5, 801, 12114, 9, 608, 98, 4, 20, 23387, 14414, 744, 5831, 1595, 727, 6, 151, 42, 186, 4, 140, 16, 67, 1462, 278, 15, 1826, 5, 3932, 17, 27, 29, 830, 632, 8825, 11, 5420, 6, 190, 25, 369, 1961, 503, 32, 3282, 1379, 59, 1078, 4, 20, 5420, 443, 34, 450, 41, 20646, 11, 34377, 1469, 19473, 1200, 11, 485, 360, 4, 140, 34, 67, 57, 35188, 7, 6654, 39, 8442, 10881, 6, 39, 2270, 5448, 9, 9809, 19, 2732, 8, 16687, 39, 1579, 4, 20, 394, 17, 27, 29, 637, 165, 34, 341, 5, 1061, 7, 33656, 414, 31, 10703, 61, 16, 341, 7, 1004, 66, 39, 1983, 4, 20

In [12]:
train_data = Articles("../data/final-data/debugdata/train_basic.json")
val_data = Articles("../data/final-data/debugdata/eval_basic.json")

In [13]:
for idx in range(len(train_data)):
    train_data.examples[idx]['text'] = tokenizer.tokenize(train_data.examples[idx]['text'])
    if len(train_data.examples[idx]['text']) > 512:
        train_data.examples[idx]['text'] = train_data.examples[idx]['text'][:512]
    train_data.examples[idx]['text'] = tokenizer.encode(
                        train_data.examples[idx]['text'],           
                        add_special_tokens = True, 
                        max_length = 512)
    train_data.examples[idx]['model_publication'] = 1 if train_data.examples[idx]['model_publication'] == 'target' else 0

In [10]:
#Create batches with positive samples in first half and negative examples in second half
class BatchSamplerWithNegativeSamples(torch.utils.data.Sampler):
    def __init__(self, pos_sampler, neg_sampler, batch_size, items):
        self._pos_sampler = pos_sampler
        self._neg_sampler = neg_sampler
        self._items = items
        assert batch_size % 2 == 0, 'Batch size must be divisible by two for negative samples.'
        self._batch_size = batch_size
        
    def __iter__(self):
        batch, neg_batch = [], []
        neg_sampler = iter(self._neg_sampler)
        for pos_idx in self._pos_sampler:
            batch.append(pos_idx)
            neg_idx = pos_idx
            # keep sampling until we get a true negative sample
            while self._items[neg_idx] == self._items[pos_idx]:
                try:
                    neg_idx = next(neg_sampler)
                except StopIteration:
                    neg_sampler = iter(self._neg_sampler)
                    neg_idx = next(neg_sampler)
            neg_batch.append(neg_idx)
            if len(batch) == self._batch_size // 2:
                batch.extend(neg_batch)
                yield batch
                batch, neg_batch = [], []
        return

    def __len__(self):
        return len(self._pos_sampler) // self._batch_size

In [11]:
#define function to return necessary data for dataloader to pass into model
def collate_fn(examples):
    words = []
    articles = []
    labels = []
    publications = []
    for example in examples:
        words.append(example['text'])
        articles.append(example['url'])
        labels.append(example['model_publication'])
        publications.append(example['publication'])
    num_words = [len(x) for x in words]
    words = np.concatenate(words, axis=0)
    word_attributes = torch.tensor(words, dtype=torch.long)
    articles = torch.tensor(articles, dtype=torch.long)
    num_words.insert(0,0)
    num_words.pop(-1)
    attribute_offsets = torch.tensor(np.cumsum(num_words), dtype=torch.long)
    publications = torch.tensor(publications, dtype=torch.long)
    real_labels = torch.tensor(labels, dtype=torch.long)
    return publications, articles, word_attributes, attribute_offsets, real_labels

In [15]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

100%|████████████████████████████████████████████████████████████████| 440473133/440473133 [01:37<00:00, 4499883.22B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# import necessary libraries
import pandas as pd
import re
import torch
import collections
import numpy as np
import json
import time
import torch.nn as nn
import os
import argparse
import arguments.train_arguments as arguments
from data_processing.articles import Articles
from models.models import InnerProduct
import data_processing.dictionaries as dictionary
import sampling.sampler_util as sampler_util
import training.eval_util as eval_util
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from transformers import BertTokenizer

